# 02 — Inférence & Évaluation (TFLite)

- Charge des modèles **TFLite** déjà fournis (ex: SSD MobileNet v2, tiny YOLO nano, YOLOv8n quant.).
- Fait l'inférence sur l'ensemble **test** (classe unique: *frelon*).
- Calcule Precision/Recall/mAP (COCO-like) pour cette classe uniquement.
- Visualise quelques prédictions.

In [20]:
import numpy as np, json, cv2, os, itertools, math
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

DATA_DIR = Path("hornet-bees-2")  # dossier créé par Roboflow download
assert DATA_DIR.exists(), f"{DATA_DIR} introuvable. Exécutez d'abord 01_data.ipynb."

TEST_DIR = DATA_DIR / "test"
TEST_ANN = TEST_DIR / "_annotations_frelon_single.coco.json"
with open(TEST_ANN, "r", encoding="utf-8") as f:
    coco = json.load(f)

id_to_img = {im["id"]: im for im in coco["images"]}
gts_by_img = {}
for a in coco["annotations"]:
    gts_by_img.setdefault(a["image_id"], []).append(a)

print("Images test:", len(coco["images"]))
print("Annotations test:", coco["annotations"])

Images test: 45
Annotations test: [{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [113, 205, 36.5, 74.5], 'area': 2719.25, 'segmentation': [], 'iscrowd': 0}, {'id': 3, 'image_id': 3, 'category_id': 1, 'bbox': [369, 149, 45, 38.5], 'area': 1732.5, 'segmentation': [], 'iscrowd': 0}, {'id': 6, 'image_id': 6, 'category_id': 1, 'bbox': [159, 179, 41.5, 69.5], 'area': 2884.25, 'segmentation': [], 'iscrowd': 0}, {'id': 7, 'image_id': 6, 'category_id': 1, 'bbox': [228, 372, 45.5, 43.5], 'area': 1979.25, 'segmentation': [], 'iscrowd': 0}, {'id': 8, 'image_id': 7, 'category_id': 1, 'bbox': [222, 128, 31, 48.5], 'area': 1503.5, 'segmentation': [], 'iscrowd': 0}, {'id': 9, 'image_id': 7, 'category_id': 1, 'bbox': [157, 208, 39, 48.5], 'area': 1891.5, 'segmentation': [], 'iscrowd': 0}, {'id': 10, 'image_id': 8, 'category_id': 1, 'bbox': [114, 229, 38, 58], 'area': 2204, 'segmentation': [], 'iscrowd': 0}, {'id': 12, 'image_id': 10, 'category_id': 1, 'bbox': [43, 262, 49, 54.5], 'area': 2670.5, '

## Utilitaires TFLite (loader générique + post-process naïf NMS)

In [14]:
def load_tflite_interpreter(tflite_path):
    interpreter = tf.lite.Interpreter(model_path=str(tflite_path))
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    return interpreter, input_details, output_details

def preprocess_image(path, input_size):
    img = cv2.imread(str(path))
    h0, w0 = img.shape[:2]
    img_resized = cv2.resize(img, input_size, interpolation=cv2.INTER_LINEAR)
    # Normalisation simple 0-255 -> 0-1 (change if model expects something else)
    img_float = img_resized.astype(np.float32) / 255.0
    img_batched = np.expand_dims(img_float, 0)
    return img, (h0, w0), img_batched

def nms(boxes, scores, iou_thr=0.5, top_k=100):
    # boxes: [N,4] in xyxy
    idxs = scores.argsort()[::-1]
    keep = []
    while idxs.size > 0 and len(keep) < top_k:
        i = idxs[0]
        keep.append(i)
        if idxs.size == 1: break
        ious = []
        for j in idxs[1:]:
            xx1 = max(boxes[i,0], boxes[j,0])
            yy1 = max(boxes[i,1], boxes[j,1])
            xx2 = min(boxes[i,2], boxes[j,2])
            yy2 = min(boxes[i,3], boxes[j,3])
            inter = max(0, xx2-xx1) * max(0, yy2-yy1)
            area_i = (boxes[i,2]-boxes[i,0])*(boxes[i,3]-boxes[i,1])
            area_j = (boxes[j,2]-boxes[j,0])*(boxes[j,3]-boxes[j,1])
            union = area_i + area_j - inter + 1e-6
            ious.append(inter/union)
        idxs = idxs[1:][np.array(ious) <= iou_thr]
    return np.array(keep, dtype=int)

## Adaptateurs simples par modèle (à ajuster selon tes exports)

In [15]:
from typing import List, Tuple

def infer_ssd_mobilenet(interpreter, input_details, output_details, img_batched, orig_size):
    # Hypothèse TFOD standard: outputs: boxes [1,N,4] in y1,x1,y2,x2 normalized, scores [1,N], classes [1,N]
    interpreter.set_tensor(input_details[0]["index"], img_batched.astype(np.float32))
    interpreter.invoke()
    boxes = interpreter.get_tensor(output_details[0]["index"])[0]  # [N,4]
    classes = interpreter.get_tensor(output_details[1]["index"])[0] # [N]
    scores = interpreter.get_tensor(output_details[2]["index"])[0]  # [N]
    h, w = orig_size
    # convert to xyxy pixel
    xyxy = np.stack([boxes[:,1]*w, boxes[:,0]*h, boxes[:,3]*w, boxes[:,2]*h], axis=1)
    return xyxy, scores, classes

def infer_yolo_like(interpreter, input_details, output_details, img_batched, orig_size):
    # Très dépendant du modèle; ici on suppose une sortie unique [1,Num,6] (x,y,w,h,conf,cls)
    interpreter.set_tensor(input_details[0]["index"], img_batched.astype(np.float32))
    interpreter.invoke()
    out = []
    for od in output_details:
        out.append(interpreter.get_tensor(od["index"]))
    # concat sur l'axe des anchors si besoin
    pred = np.concatenate([o.reshape(1,-1,o.shape[-1]) for o in out], axis=1)[0]  # [N,6] or [N,??]
    # Try to parse x,y,w,h,conf,cls
    if pred.shape[-1] < 6:
        raise ValueError("Sortie YOLO inattendue; ajuste 'infer_yolo_like'.")
    x, y, w, h, conf, cls = [pred[:,i] for i in range(6)]
    # Convert cxcywh -> xyxy
    cx, cy = x, y
    x1 = (cx - w/2.0) * orig_size[1]
    y1 = (cy - h/2.0) * orig_size[0]
    x2 = (cx + w/2.0) * orig_size[1]
    y2 = (cy + h/2.0) * orig_size[0]
    xyxy = np.stack([x1,y1,x2,y2], axis=1)
    scores = conf
    classes = cls
    return xyxy, scores, classes

## Évaluation (AP/AR simplifiés pour 1 classe)

In [16]:
def iou_xyxy(a, b):
    xx1 = max(a[0], b[0]); yy1 = max(a[1], b[1])
    xx2 = min(a[2], b[2]); yy2 = min(a[3], b[3])
    inter = max(0, xx2-xx1) * max(0, yy2-yy1)
    area_a = (a[2]-a[0])*(a[3]-a[1])
    area_b = (b[2]-b[0])*(b[3]-b[1])
    return inter / (area_a + area_b - inter + 1e-6)

def eval_detections(preds_by_img, gts_by_img, iou_thr=0.5):
    # preds_by_img: dict[img_id] -> list of (xyxy, score)
    # gts_by_img: dict[img_id] -> list of ann with 'bbox' [x,y,w,h]
    all_scores, all_matches = [], []
    n_gt = 0
    for img_id, gts in gts_by_img.items():
        n_gt += len(gts)
        gts_used = [False]*len(gts)
        preds = preds_by_img.get(img_id, [])
        preds = sorted(preds, key=lambda x: x[1], reverse=True)
        for (box, s) in preds:
            match = 0
            for i,gt in enumerate(gts):
                gx,gy,gw,gh = gt["bbox"]
                gxyxy = [gx,gy,gx+gw,gy+gh]
                if not gts_used[i] and iou_xyxy(box, gxyxy) >= iou_thr:
                    gts_used[i] = True
                    match = 1
                    break
            all_scores.append(s)
            all_matches.append(match)
    # Compute Precision-Recall curve + AP (11-point)
    order = np.argsort(-np.array(all_scores))
    tp = np.cumsum(np.array(all_matches)[order])
    fp = np.cumsum(1 - np.array(all_matches)[order])
    recalls = tp / max(n_gt, 1)
    precisions = tp / np.maximum(tp+fp, 1e-9)
    # 11-point interpolated AP
    ap = 0.0
    for r in np.linspace(0,1,11):
        p = 0.0
        mask = recalls >= r
        if mask.any():
            p = np.max(precisions[mask])
        ap += p/11.0
    return {"AP@0.5": float(ap), "nGT": int(n_gt)}

## Exécuter l'inférence pour chaque modèle disponible

In [17]:
MODELS_DIR = Path("../src/models")
candidates = {
    "ssd_mobilenet": MODELS_DIR / "ssd_mobilenet_v2_fpnlite_035_416_int8.tflite",
    "tiny_yolo_nano": MODELS_DIR / "st_yolo_x_nano_416_0.33_0.25_int8.tflite",
    "yolov8n_quant": MODELS_DIR / "yolov8n_416_quant_pc_uf_od_coco-person.tflite",
}
available = {k:v for k,v in candidates.items() if v.exists()}
print("Modèles trouvés:", available)

results = {}
viz_dir = Path("predictions_viz"); viz_dir.mkdir(exist_ok=True)

# Map COCO image_id -> file path
img_path_by_id = {im["id"]: TEST_DIR / im["file_name"] for im in coco["images"]}

for name, path in available.items():
    print(f"==> {name}:", path)
    interpreter, inp, out = load_tflite_interpreter(path)
    # derive input size
    _, in_h, in_w, _ = inp[0]["shape"]
    preds_by_img = {}
    for img_id, im_meta in tqdm(id_to_img.items(), total=len(id_to_img)):
        if (TEST_DIR / im_meta["file_name"]).exists():
            img_bgr, (h0,w0), img_batched = preprocess_image(TEST_DIR / im_meta["file_name"], (in_w, in_h))
            try:
                if "ssd_mobilenet" in name:
                    xyxy, scores, classes = infer_ssd_mobilenet(interpreter, inp, out, img_batched, (h0,w0))
                else:
                    xyxy, scores, classes = infer_yolo_like(interpreter, inp, out, img_batched, (h0,w0))
            except Exception as e:
                # skip image if incompatible
                continue
            # garder uniquement frelon (cls 0/1 varie selon modèle; ici on prend tout et NMS)
            keep = scores > 0.25
            xyxy = xyxy[keep]; scores_f = scores[keep]
            if xyxy.size:
                k = nms(xyxy, scores_f, iou_thr=0.5, top_k=50)
                preds_by_img[img_id] = [(xyxy[i], float(scores_f[i])) for i in k]
            else:
                preds_by_img[img_id] = []
    metrics = eval_detections(preds_by_img, gts_by_img, iou_thr=0.5)
    results[name] = metrics
    print(name, metrics)

results

Modèles trouvés: {'ssd_mobilenet': WindowsPath('../src/models/ssd_mobilenet_v2_fpnlite_035_416_int8.tflite'), 'tiny_yolo_nano': WindowsPath('../src/models/st_yolo_x_nano_416_0.33_0.25_int8.tflite'), 'yolov8n_quant': WindowsPath('../src/models/yolov8n_416_quant_pc_uf_od_coco-person.tflite')}
==> ssd_mobilenet: ..\src\models\ssd_mobilenet_v2_fpnlite_035_416_int8.tflite


100%|██████████| 45/45 [00:00<00:00, 149.21it/s]


ssd_mobilenet {'AP@0.5': 0.0, 'nGT': 53}
==> tiny_yolo_nano: ..\src\models\st_yolo_x_nano_416_0.33_0.25_int8.tflite


100%|██████████| 45/45 [00:00<00:00, 219.05it/s]


tiny_yolo_nano {'AP@0.5': 0.0, 'nGT': 53}
==> yolov8n_quant: ..\src\models\yolov8n_416_quant_pc_uf_od_coco-person.tflite


100%|██████████| 45/45 [00:00<00:00, 187.85it/s]

yolov8n_quant {'AP@0.5': 0.0, 'nGT': 53}


{'ssd_mobilenet': {'AP@0.5': 0.0, 'nGT': 53},
 'tiny_yolo_nano': {'AP@0.5': 0.0, 'nGT': 53},
 'yolov8n_quant': {'AP@0.5': 0.0, 'nGT': 53}}

## Visualisation qualitative

In [18]:
def draw_boxes(img, boxes, color=(0,255,0)):
    for b in boxes:
        x1,y1,x2,y2 = map(int, b)
        cv2.rectangle(img, (x1,y1), (x2,y2), color, 2)
    return img

# Afficher quelques images avec prédictions du meilleur modèle (AP max)
best = max(results.items(), key=lambda kv: kv[1]["AP@0.5"])[0] if results else None
print("Best model:", best)
if best:
    # Recharge interpréteur pour dessiner 8 images
    interpreter, inp, out = load_tflite_interpreter(candidates[best])
    _, in_h, in_w, _ = inp[0]["shape"]
    shown = 0
    for img_id, meta in id_to_img.items():
        p = TEST_DIR / meta["file_name"]
        if not p.exists(): continue
        img_bgr, (h0,w0), img_batched = preprocess_image(p, (in_w, in_h))
        try:
            if "ssd_mobilenet" in best:
                xyxy, scores, classes = infer_ssd_mobilenet(interpreter, inp, out, img_batched, (h0,w0))
            else:
                xyxy, scores, classes = infer_yolo_like(interpreter, inp, out, img_batched, (h0,w0))
        except:
            continue
        keep = scores > 0.25
        xyxy = xyxy[keep]
        if xyxy.size:
            img_draw = draw_boxes(img_bgr.copy(), xyxy)
            img_rgb = cv2.cvtColor(img_draw, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(6,6))
            plt.imshow(img_rgb); plt.axis("off"); plt.title(f"{best}")
            plt.show()
            shown += 1
        if shown >= 8: break

Best model: ssd_mobilenet


In [ ]:
# Train

